# NOC processing
#### This is the fourth and final step of the enrichment calculation
Calculate NOC proportion and append to the enrichment table  

Key output files
- `yyyy-mm-dd_enrichment_table_NOC_prop.csv`: enrichment values and NOC proportion  
- `yyyy-mm-dd_volcano_table.csv`: enrichment values and p-values  
  
both files are in the directory `output/enrichment_and_volcano_tables`

In [1]:
import os
import sys
from datetime import datetime
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd

script_path = Path.cwd().parent.parent / "script"
data_path = Path.cwd().parent.parent / "data"
sys.path.append(str(script_path))

from pyseus import basic_processing as ip
from pyseus import contrast_tools as ct

### Load imputed NOC/Bulk table
The correct datestamp is required to find the input files

In [2]:
# %store -r timestamp USE_FROZEN
# if USE_FROZEN:
#     raise Exception("USE_FROZEN is true, you probably want to skip enrichment and proceed from Fig1")
timestamp = "2024-07-24"
print(f"Timestamp: {timestamp}")

outprefix = f"{timestamp}_QC_filter_impute"

outdir = Path.cwd() / "output"
preprocessing_out = outdir / f"preprocessing"

NOC_path = preprocessing_out / f"{outprefix}_filtered_table.csv"

try:
    noc_table = pd.read_csv(NOC_path, header=[0], index_col=0)
except FileNotFoundError:
    print(f"File {NOC_path} not found.\nPlease rerun the previous steps or specify the correct timestamp, current value is {timestamp}")
except pd.errors.ParserError:
    print(f"There was an error parsing the CSV file at {NOC_path}.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Timestamp: 2024-07-24


### Calculation of proportion

In [3]:
# take only the noc columns
NOC_kw = ["Cytosol", "Nuclear", "Organelle"]
noc_cols = [
    col for col in list(noc_table)
    if (any(kw in col for kw in NOC_kw)) and "Proteome" not in col
]
print(noc_cols)

['UnInfected_Cytosol_1', 'UnInfected_Cytosol_2', 'UnInfected_Cytosol_3', 'UnInfected_Nuclear_1', 'UnInfected_Nuclear_2', 'UnInfected_Nuclear_3', 'UnInfected_Organelle_1', 'UnInfected_Organelle_2', 'UnInfected_Organelle_3']


In [4]:
# just select the noc_cols + Protein IDs for merge purpose
noc_table = noc_table[['Protein IDs'] + noc_cols].copy()

In [5]:
# group the tables and take the median

# start RawTables class from basic processing to group the samples
noc_process = ip.RawTables(proteingroup=None, file_designated=True,
    info_cols=['Protein IDs'], sample_cols=noc_cols)

# designate the pre-computed noc filtered table on this class
noc_process.filtered_table = noc_table

# transform the intensity to linear space
#noc_process.transform_intensities(func=np.exp2)

# group replicates
noc_process.group_replicates(reg_exp=r'(.*)_\d+$')

# calculate the median
noc_medians = ip.median_replicates(noc_process.grouped_table)
noc_medians.head()

Intensity values have not been transformed yet from filtered table,
we recommend using transform_intensities() method before grouping replicates.

Using filtered_table to group replicates.


,UnInfected_Organelle,UnInfected_Nuclear,UnInfected_Cytosol,Protein IDs
2,76811000.0,166520000.0,75060000.0,A0A023T6R1;Q96A72;F5H6P7;F5H6N1;F5H3U9;F5H124
3,194700000.0,238020000.0,245940000.0,Q9Y5S9;A0A023T787;A0A0J9YW13
5,51652000.0,22978000.0,4330700.0,A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6Z...
6,0.0,0.0,3272600.0,A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...
7,77290000.0,24210000.0,9258400.0,Q99805;A0A024QYR8;B3KSG9


In [6]:
# we then take the sum of each column 
noc_medians['noc_sum_uninf'] = noc_medians['UnInfected_Cytosol'] + noc_medians['UnInfected_Organelle'] + noc_medians['UnInfected_Nuclear']

# remove all proteingroups where both inf and uninf noc_sum is 0 -> there is no information on these
noc_medians = noc_medians[(noc_medians['noc_sum_uninf'] > 0)].reset_index(drop=True)

# create a new dataframe with just the proportion: intensity divided by the sum
noc_proportion = noc_medians[['Protein IDs']].copy()
noc_proportion['NOC_cytosol'] = noc_medians['UnInfected_Cytosol'] / noc_medians['noc_sum_uninf']
noc_proportion['NOC_organelle'] = noc_medians['UnInfected_Organelle'] / noc_medians['noc_sum_uninf']
noc_proportion['NOC_nuclear'] = noc_medians['UnInfected_Nuclear'] / noc_medians['noc_sum_uninf']


### Merge NOC proportion with IP enrichment

In [7]:
# merge with the enrichment table
# read the enrichment table
enrich_out_dir = outdir / "enrichment_and_volcano_tables"
enrichment_csv_path = enrich_out_dir / f"{timestamp}_enrichment_table.csv"
enrichments = pd.read_csv(enrichment_csv_path, header=[0, 1], index_col=0)
meta_cols = enrichments["metadata"].columns
enrichments = enrichments.droplevel(0, axis=1)

In [8]:
# finally, merge with the enrichment table
enrichment_nocs = enrichments.merge(noc_proportion, on="Protein IDs", how="inner")

# standard formatting
enrichment_nocs = ct.standard_pyseus_headers(enrichment_nocs, meta_cols=meta_cols)

In [9]:
# save the enrichment table with NOC proportions
enrichment_NOC_csv_path = enrich_out_dir / f"{timestamp}_enrichment_table_NOC_prop.csv"
enrichment_nocs.to_csv(enrichment_NOC_csv_path)

In [10]:
# enrichment table with NOC proportions
enrichment_nocs

metadata  \
                                            Protein IDs   
0         A0A023T6R1;Q96A72;F5H6P7;F5H6N1;F5H3U9;F5H124   
1                          Q9Y5S9;A0A023T787;A0A0J9YW13   
2     A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6Z...   
3     A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...   
4                              Q99805;A0A024QYR8;B3KSG9   
...                                                 ...   
8535                                             X5D7P8   
8536                                             X5D8X9   
8537  X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...   
8538                                             X5DQZ7   
8539                                             X6RHX1   

                                                                          \
                                   Majority protein IDs       Gene names   
0                       A0A023T6R1;Q96A72;F5H6P7;F5H6N1  FLJ10292;MAGOHB   
1                                     Q9Y5S9;A0A023T787       RBM8A;RBM8   
2     A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6ZTK5           TM9SF4   
3     A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...             PTEN   
4                              Q99805;A0A024QYR8;B3KSG9           TM9SF2   
...                                                 ...              ...   
8535                                             X5D7P8            STK39   
8536                                             X5D8X9          CNTNAP2   
8537  X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...            CRMP1   
8538                                             X5DQZ7             GPX1   
8539                                             X6RHX1            SCYL3   

        sample                                                              \
      12-LAMP1  13-RAB1A     12-WT 12-TOMM20 12-PNPLA2 12-SEC61B   12-ACTB   
0    -0.141427  0.164993 -0.403814 -0.279302 -0.386914  0.018797  0.288365   
1    -0.588500 -1.931700  1.457350 -2.246100  0.710800 -0.623300  0.860750   
2     3.703700  6.304299 -3.994900 -2.356032 -5.046599  4.604981 -4.326681   
3     0.261350  0.521100 -0.889284  0.744454  0.087087  0.297697 -0.356495   
4     5.499848  7.657225 -5.625948 -1.099823 -8.018554  6.447580 -7.542662   
...        ...       ...       ...       ...       ...       ...       ...   
8535 -0.030088 -0.838131 -0.750013 -0.523289 -0.998200  0.547829  0.125618   
8536 -0.052453 -0.518175  0.122136 -0.658738  0.072154  0.028252  0.861863   
8537  0.327787  0.309068 -0.205879  0.558718 -0.161728 -0.120634  0.805817   
8538  0.261512 -0.491176 -0.488960  2.629682 -0.817804 -0.382702 -0.287952   
8539  0.746831 -1.629260 -0.200211 -0.430547  0.021381 -0.954216 -0.148533   

      ...                                                              \
      ... 11-CEP350 09-ATG101  10-EXOC2 10-TOMM20     10-WT     09-WT   
0     ...  0.212616  3.018858 -1.121111 -1.547331  2.958600  2.270600   
1     ...  0.895300 -0.728450  1.017250 -2.969400  1.972000  1.836900   
2     ...  0.397300  2.393000 -4.497900 -2.685600  0.006200  0.315600   
3     ... -0.067346 -0.332397 -0.818591  0.223960 -0.236051  0.287790   
4     ... -0.438800 -0.412350  0.033350 -2.382900 -0.608600 -0.744500   
...   ...       ...       ...       ...       ...       ...       ...   
8535  ... -0.227490  0.874124 -0.075496  0.660835 -0.499104  0.762239   
8536  ...  1.758580  0.498581  0.379988 -0.680795  1.104980  0.309999   
8537  ... -0.334151 -0.265872  0.229145 -0.425032  0.796492 -1.147851   
8538  ... -0.481100 -0.570653  0.151380  2.896576 -1.193333 -1.834043   
8539  ... -0.767265 -0.747004  0.199421 -1.196105 -0.145880 -0.882266   

                                                      
      10-AP2B1 NOC_cytosol NOC_organelle NOC_nuclear  
0    -0.353301    0.235748      0.241247    0.523005  
1    -0.336600    0.362391      0.286889    0.350721  
2     0.620900    0.054846      0.654148    0.291006  
3    -0.083505    1.000000      0.000000    0.0000